In [ ]:
from edgar_functions import *
from scorecard_naming import Balance_sheet_mapping, Income_statement_mapping, Cash_flow_mapping
ticker = "wsm"

In [ ]:
income_sheet = form_statement_for_all_available_years(ticker, "income_statement")
balance_sheet = form_statement_for_all_available_years(ticker, "balance_sheet")
cash_flow_sheet = form_statement_for_all_available_years(ticker, "cash_flow_statement")
# income_sheet_quarterly = form_statement_for_all_available_quarters(ticker, "income_statement")
# balance_sheet_quarterly = form_statement_for_all_available_quarters(ticker, "balance_sheet")
# cash_flow_sheet_quarterly = form_statement_for_all_available_quarters(ticker, "cash_flow_statement")

In [ ]:
# acc_series = get_10K_accessionNumbers_for_ticker(ticker)
# label_dict = get_label_dictionary(ticker, acc_series[0])
# balance = rename_statement(balance_sheet, label_dict)
# income = rename_statement(income_sheet, label_dict)
# cash_flow = rename_statement(cash_flow_sheet, label_dict)
# balance = balance.drop(balance_sheet.columns[-1], axis=1)
# acc_series_quarterly = get_10Q_accessionNumbers_for_ticker(ticker)
# label_dict_quarterly = get_label_dictionary(ticker, acc_series_quarterly[0])
# balance_quarterly = rename_statement(balance_sheet_quarterly, label_dict_quarterly)
# income_quarterly = rename_statement(income_sheet_quarterly, label_dict_quarterly)
# cash_flow_quarterly = rename_statement(cash_flow_sheet_quarterly, label_dict_quarterly)

In [ ]:
def create_simple_statement(statement, mapping):
    statement = statement.T
    new_statement = pd.DataFrame(index=statement.index, columns=mapping.keys())
    for common_name, gaap_names in mapping.items():
        for gaap_name in gaap_names:
            if gaap_name in statement.columns:
                if new_statement[common_name].isnull().all():
                    new_statement[common_name] = statement[gaap_name]
                else:
                    new_statement[common_name].fillna(statement[gaap_name], inplace=True)
    
    return new_statement

In [ ]:
new_income = create_simple_statement(income_sheet, Income_statement_mapping).replace(np.nan, 0)
new_balance = create_simple_statement(balance_sheet, Balance_sheet_mapping).replace(np.nan, 0)
new_cash_flow = create_simple_statement(cash_flow_sheet, Cash_flow_mapping).replace(np.nan, 0)

In [ ]:
new_income

In [ ]:
new_balance.T

In [ ]:
new_cash_flow.T